<a href="https://colab.research.google.com/github/m-t-abbasi/SpatialPlanning/blob/main/section4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import geopandas as gpd
import numpy as np

# مسیر فایل شیپ را تعیین می کنیم
shapefile_path = "/content/landuse_shp.shp"

# خواندن فایل شیپ
gdf = gpd.read_file(shapefile_path)

# تعریف اندازه‌های مختلف بافر
buffer_sizes = [100, 200, 300]

# تعریف نام‌های کاربری برای محاسبه کوواریانس
landuse_names = gdf['landuse2'].unique()

# تعریف دیکشنری برای نگهداری میانگین کوواریانس بین دوبه دوی کاربری‌ها
mean_covariances = {}

# پیمایش تک تک اندازه‌های بافر
for buffer_size in buffer_sizes:
    # تعریف لیست برای نگهداری کوواریانس‌ها
    covariances = []
    
    # پیمایش تک تک پارسل‌های شیپ فایل
    for index, row in gdf.iterrows():
        parcel_id = row['OBJECTID']  # شماره پارسل
        
        # تعریف دیکشنری برای نگهداری کوواریانس بین دوبه دوی کاربری‌ها
        covariances_dict = {}
        
        buffer = row.geometry.buffer(buffer_size)
        
        # پیدا کردن پارسل‌هایی که کاملاً در داخل هر بافر قرار دارند
        parcels_within_buffer = gdf[gdf.geometry.within(buffer)]
        
        # شمارش تعداد پارسل‌ها به تفکیک کاربری
        landuse_counts = parcels_within_buffer['landuse2'].value_counts().to_dict()
        
        # اضافه کردن کاربری‌هایی که تعداد پارسل صفر دارند
        all_landuse_types = set(gdf['landuse2'].unique())
        for landuse_type in all_landuse_types:
            landuse_counts.setdefault(landuse_type, 0)
        
        # محاسبه مساحت هر کاربری در هر بافر
        landuse_areas = parcels_within_buffer.groupby('landuse2')['SHAPE_Area'].sum().to_dict()
        
        # محاسبه نسبت آماری برای تمامی کاربری‌ها
        for landuse_type in all_landuse_types:
            ratio = (landuse_counts.get(landuse_type, 0) * landuse_areas.get(landuse_type, 0)) / (len(parcels_within_buffer) * sum(landuse_areas.values()))
            
            # اضافه کردن نسبت آماری به دیکشنری کوواریانس
            covariances_dict[landuse_type] = ratio
        
        # اضافه کردن دیکشنری کوواریانس به لیست کوواریانس‌ها
        covariances.append(covariances_dict)
    
    # محاسبه میانگین کوواریانس بین دوبه دوی کاربری‌ها برای هر بافر
    mean_covariances[buffer_size] = {}
    for landuse_type in all_landuse_types:
        ratios = [covariances_dict[landuse_type] for covariances_dict in covariances]
        mean_ratio = np.mean(ratios)
        mean_covariances[buffer_size][landuse_type] = mean_ratio

# چاپ میانگین کوواریانس بین دوبه دوی کاربری‌ها
for buffer_size, ratios_dict in mean_covariances.items():
    print(f"میانگین کوواریانس بین دوبه دوی کاربری‌ها برای بافر {buffer_size} متری:")
    for landuse_type, mean_ratio in ratios_dict.items():
        print(f"{landuse_type}: {mean_ratio}")
    print()



میانگین کوواریانس بین دوبه دوی کاربری‌ها برای بافر 100 متری:
maskooni: 0.5436621672641153
bayer: 0.023560578385124837
sanati: 4.571579273161961e-05
tejari: 0.040645662082867955
park: 0.001274276913329715
khadamati: 0.0013557533693973381
amozshi: 0.0008148725543067538
darmani: 7.112711447806713e-05

میانگین کوواریانس بین دوبه دوی کاربری‌ها برای بافر 200 متری:
maskooni: 0.519676132416553
bayer: 0.014835791595529243
sanati: 2.1442048005719993e-05
tejari: 0.026998883958410082
park: 0.0009579079052723424
khadamati: 0.0010368350559236612
amozshi: 0.0006937134164065159
darmani: 2.2024090921289806e-05

میانگین کوواریانس بین دوبه دوی کاربری‌ها برای بافر 300 متری:
maskooni: 0.5000996461998799
bayer: 0.010086669765425077
sanati: 1.4479118153160524e-05
tejari: 0.023142016563343484
park: 0.0005920757985151497
khadamati: 0.0009351193143672815
amozshi: 0.0005625144102920566
darmani: 1.6260954357068873e-05

